In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import pickle

import matplotlib.pyplot as plt

In [3]:
# Read Excel file
df_sourcefile = pd.read_excel(
    'Data/SF_DataV2.xlsx', sheet_name=0)
# check data download type. Should consume CSV format. For new its a XLSX Format.
print("Shape of dataframe is: {}".format(df_sourcefile.shape))
df_sourcefile.head()

Shape of dataframe is: (14718, 16)


,lastName,firstName,status,username,user-id,currency-code,paycompvalue,start-date,end-date,pay-component,frequency,date-of-birth,start-date.1,end-date.1,end-date.2,fte
0,Trombley,Francois,Active User,ftrombley,1,EUR,6500.00,2010-01-30,2013-12-31 00:00:00,BASESAL_FR,MON,1978-06-05,2010-01-30,2017-04-12 00:00:00,NaT,1.0
1,Trombley,Francois,Active User,ftrombley,1,EUR,6763.26,2017-01-01,9999-12-31 00:00:00,BASESAL_FR,MON,1978-06-05,2010-01-30,2017-04-12 00:00:00,NaT,1.0
2,Trombley,Francois,Active User,ftrombley,1,EUR,6696.30,2016-01-01,2016-12-31 00:00:00,BASESAL_FR,MON,1978-06-05,2010-01-30,2017-04-12 00:00:00,NaT,1.0
3,Trombley,Francois,Active User,ftrombley,1,EUR,6630.00,2014-01-01,2015-12-31 00:00:00,BASESAL_FR,MON,1978-06-05,2010-01-30,2017-04-12 00:00:00,NaT,1.0
4,Trombley,Francois,Active User,ftrombley,1,EUR,6500.00,2010-01-30,2013-12-31 00:00:00,BASESAL_FR,MON,1978-06-05,2010-01-30,2017-04-12 00:00:00,NaT,1.0


# Data Cleaning

drop dublicate if rows are identical

In [25]:
df = df_sourcefile.copy()
#df.drop_duplicates(subset=None, keep='first', inplace=False)
df.sort_values(by=['user-id'], ascending=[0])
print("Shape of dataframe is: {}".format(df.shape))
#df['user-id'][0]
df.head()

Shape of dataframe is: (14718, 16)


,lastName,firstName,status,username,user-id,currency-code,paycompvalue,start-date,end-date,pay-component,frequency,date-of-birth,start-date.1,end-date.1,end-date.2,fte
0,Trombley,Francois,Active User,ftrombley,1,EUR,6500.00,2010-01-30,2013-12-31 00:00:00,BASESAL_FR,MON,1978-06-05,2010-01-30,2017-04-12 00:00:00,NaT,1.0
1,Trombley,Francois,Active User,ftrombley,1,EUR,6763.26,2017-01-01,9999-12-31 00:00:00,BASESAL_FR,MON,1978-06-05,2010-01-30,2017-04-12 00:00:00,NaT,1.0
2,Trombley,Francois,Active User,ftrombley,1,EUR,6696.30,2016-01-01,2016-12-31 00:00:00,BASESAL_FR,MON,1978-06-05,2010-01-30,2017-04-12 00:00:00,NaT,1.0
3,Trombley,Francois,Active User,ftrombley,1,EUR,6630.00,2014-01-01,2015-12-31 00:00:00,BASESAL_FR,MON,1978-06-05,2010-01-30,2017-04-12 00:00:00,NaT,1.0
4,Trombley,Francois,Active User,ftrombley,1,EUR,6500.00,2010-01-30,2013-12-31 00:00:00,BASESAL_FR,MON,1978-06-05,2010-01-30,2017-04-12 00:00:00,NaT,1.0


FTE should be lower equal 1

In [ ]:
if df['fte'] > 1:
    #df.defete...

calculate daily rate

In [ ]:
# case clause einbauen für pay-component varianten, um auf die Daily Rate zu kommen
if df['pay-component'] = "BASESAL_*":
    # 4 Week with 5 working days for one FTE
    df['daily-rate'] = ( df['paycompvalue'] / (4 * 5) ) * df['fte']
    # usw....